<a href="https://colab.research.google.com/github/lim-jr/ESAA/blob/main/0410%EC%9B%94%EA%B3%BC%EC%A0%9C_%EC%99%84%EB%B2%BD%EA%B0%80(487_496)_%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%B6%84%EB%A5%98%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CH8 텍스트 분석

## 8.3 텍스트 분류 실습 -20 뉴스그룹 분류 
* 텍스트 분류는 특정 문서의 분류를 학습 데이터를 통해 학습해 모델을 생성한 뒤    
  이 학습 모델을 이용해 다른 문서의 분류를 예측하는 것임. 
1. 텍스트를 정규화
2. 피처 벡터화     
  텍스트를 피처벡터화로 변환하면 희소행렬 형태
3. 알고리즘 적용해 분류를 학습/예측/평가    
  희소행렬에 분류를 잘 처리할 수 있는 알고리즘은 로지스틱 회귀, 선형서포트벡터머신, 나이브베이즈
4. 파라미터튜닝, Pipeline

### 8.3.1 텍스트 정규화 

In [3]:
from sklearn.datasets import fetch_20newsgroups
news_data = fetch_20newsgroups(subset='all', random_state=156)
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


* fetch_20newgroups()는 bunch 객체를 반환

In [4]:
import pandas as pd 
print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


* 타겟 클래스 값은 0~19까지 20개로 구성돼 있음.


In [5]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

* 뉴스그룹 기사의 내용뿐 아니라 뉴스그룹 제목, 작성자, 소속, 이메일 등 다양한 정보를 가지고 있다. 
* 이 중에서 내용을 제외하고 제목등의 다른 정보는 제외한다.   
  왜냐면 제목, 소속, 이메일주소 등의 헤더와 푸터 정보들은 뉴스그룹 분류 Target 클래스 값과 유사한 데이터가를 가지고 있는 경우가 너무 많아서.
* 순수한 텍스트만으로 구성된 기사 내용으로 어떤 뉴스그룹에 속하는지 분류류

In [9]:
from sklearn.datasets import fetch_20newsgroups

# subset='train' 으로 학습용 데이터만 추출, remove('header', 'footers', 'quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotoes'),
                   random_state=156)
X_train = train_news.data
y_train = train_news.target

# subset='test' 으로 학습용 데이터만 추출, remove('header', 'footers', 'quotes')로 내용만 추출
test_news = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotoes'),
                   random_state=156)
X_test = test_news.data
y_test = test_news.target

print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data), len(test_news.data)))
len(test_news.data)

학습 데이터 크기 11314, 테스트 데이터 크기 7532


7532

### 8.3.2 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가
* fit -> transform (o)    
  fit_transform (x)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization 으로 피처 벡터화 변환 수행 
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습데이터로 fit()된 CountVectorizer를 이용해 테스트 데이터를 피처벡터화 변환 수행
X_test_cnt_vect = cnt_vect.transform(X_test)
print('학습데이터 텍스트의 CountVectorizer Shape:', X_train_cnt_vect.shape)

학습데이터 텍스트의 CountVectorizer Shape: (11314, 114751)


* 11314개의 문서에서, 피처(단어)가 114751개로 만들어짐.


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression 을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.709


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


TF-IDF가 단순 카운트 기반보다 훨씬 높은 예측 정화도를 제공한다. 일반적으로 문서 내에 특스트가 많고 많은 문서를 가지는 텍스트 분석에서 카운트 벡터화보다는 TF-IDF벡터화가 좋은 예측 결과를 도출한다.

* 텍스트 분석 성능 향상 방법
1. 최적의 ml알고리즘을 선택하는 것
2. 최상의 피처 전처리를 수행하는 것

In [ ]:
# stop words 필터링을 추가하고 ngram을 기본 (1,1)에서 (1,2)로 변경해 피처 벡터화 적용
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidt_vect = tfidf_vect.transform(X_train)
X_test_tfidt_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidt_vect, y_train)
pred = lr_clf.predict(X_test_tfidt_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.956161


In [ ]:
from sklearn.model_selection import GridSearchCV

# 최적 C값 도출 튜닝 수행. CV는 3폴드 세트로 설정
params = { 'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(X_train_tfidt_vect, y_train)
print('Logistic Regression best C parameter :', grid_cv_lr.best_params_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

Logistic Regression best C parameter : {'C': 10}


In [ ]:
# 최적 C값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidt_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.972865


## 사이킷런 파이프라인 사용 및 gridsearchCV와의 결합

Pipeline을 이용하면 피처 벡터화와 ml알고리즘 학습/예측을 위한 코드 작성을 한 번에 진행할 수 있다. 

텍스트 기반의 피처 벡터화뿐만 아니라 모든 데이터 전처리 작업과 estimator를 결합할 수 있다. 

In [ ]:
from sklearn.pipeline import Pipeline

# TfidVectorizer 객체를 tfid_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline 생성
pipeline=Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
    ('lr_clf', LogisticRegression(C=10))
])

# 별도의 TfidVectorizer 객체의 fit(), transform()과 LogisticRegression의 fit(), predict()가 필요없음
# pipeline의 fit과 predict()만으로 한꺼번에 피처 벡터화와 ML 학습/예측이 가능
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:3f}'.format(accuracy_score(y_test, pred)))

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline을 통한 Logistic Regression의 예측 정확도는 0.972865


피처벡터화를 위한 파라미터와 ML알고리즘의 하이퍼 파라미터를 모두 한번에 튜닝할 수 있다. 

estimator가 아닌 pipeline을 입력할 경우에는 param_grid의 입력 값 설정이 기존과 다르다.

피처 벡터화 객체의 파라미터와 Estimator객체의 하이퍼 파라미터를 각각 구별할 수 있어야하는데 , 이때 개별 객체 명과 파라미터명/하이퍼 파라미터명을 결합해 key값으로 할당한다.

In [ ]:
from sklearn.model_selection import GridSearchCV

pipeline=Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(__) 2개를 연달아 붙여 GridSearchCV에 사용될 파라미터/하이퍼파라미터 이름과 값을 설정
params = {'tfidf_vect__ngram_range':[(1,1),(1,2),(1,3)],
         'tfidf_vect__max_df': [100,300,700],
         'lr_clf__C':[1,5,10]}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}.format(accuracy_score(y_test,pred))')

Fitting 3 folds for each of 27 candidates, totalling 81 fits
